In [4]:
from dotenv import load_dotenv
import os

import ollama
import prompt
import requests
import runpod
import os

c:\Users\user\anaconda3\envs\Gameprj\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
load_dotenv()

runpod.api_key = os.getenv('RUNPOD_API_KEY')
pod_id = os.getenv("POD_ID")

In [ ]:
def query_ollama(prompt: str, pod_id: str, model: str = "EEVE-Korean-10.8B") -> str:

    if not pod_id:
        raise ValueError("pod_id를 지정해주세요.")

    # GPU Pod 내부에서 실행할 코드 (Ollama API 호출)
    code = f"""
        import ollama
        response = ollama.chat(
            model='{model}',
            messages=[{{"role": "user", "content": {repr(prompt)}}}]
        )
        print(response)
        """

    # RunPod SDK로 Pod에서 코드 실행
    job = runpod.api.run(pod_id=pod_id, input={"script": code})

    # 실행 결과 추출
    output = job.get("output", "")
    return output.strip()